<a href="https://colab.research.google.com/github/AlanDarioMoreno/Brisa-Notebooks-Modelos-IA/blob/Resnet50-5clases%2FRetino-NoRetino/Entrenamiento_modelo_Resnet50_5Clases_Retino_NoRetino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Se vincula Google Drive para hacer la precarga de modelos y utilizar los datasets generados
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ssl
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models


# Contantes que utilizamos para manejar los distintos tipos de entrenamientos
IMAGE_SIZE= 512
EPOCS = 4
LR = 1e-5
BATCH_SIZE = 64
NUM_CLASES = 2

# Se define la capa GeM que reemplazara al avgPooling de Resnet50
def gem(x, p=3, eps=1e-6):
    return nn.functional.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return gem(x, p=self.p, eps=self.eps)

# Declara un PATH para el guardado del entrenamiento o el re-entrenamiento de otro modelo
model_path ='/content/drive/MyDrive/Colab/ResNet50/RetinaNoRetina_resnet50_v1.pth'

# Cargar el modelo ResNet50 con la capa GeM ya implementada
model = models.resnet50(pretrained=False)
model.avgpool = GeM()

# Ajusta  la cantidad de capas de salida del modelo definido con la constante NUM_CLASES
model.fc = nn.Linear(model.fc.in_features, NUM_CLASES)

# Intentar cargar el modelo si existe
try:
    model.load_state_dict(torch.load(model_path))
    print("Modelo cargado exitosamente.")
except FileNotFoundError:
    print("Modelo no encontrado, entrenando desde cero.")

# Transformacio de imagen
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Cargar el dataset
image_dir = '/content/drive/MyDrive/01Retina-NoRetina'
dataset = datasets.ImageFolder(root=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Configuración del dispositivo y criterio de pérdida
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR) # Aqui se aplica el Learning Rate definido por constante. Se fue modificando dependiendo los resultados de los entrenamientos
model.to(device)

# Función para calcular precisión
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    corrects = torch.sum(preds == labels).item()
    return corrects / labels.size(0)

# Inicializar la mejor pérdida como un valor alto al principio
best_loss = float('inf')

# Entrenamiento
num_epochs = EPOCS
for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')
    running_loss = 0.0
    running_accuracy = 0.0
    total_batches = 0

    model.train()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calcular precisión y pérdida para el batch actual
        batch_accuracy = calculate_accuracy(outputs, labels)
        running_loss += loss.item()
        running_accuracy += batch_accuracy
        total_batches += 1

        # Mostrar precisión y pérdida para cada batch
        print(f"Lote {total_batches} - Loss: {loss.item():.4f} - Accuracy: {batch_accuracy * 100:.2f}%")

    # Promedio de pérdida y precisión por EPOC
    epoch_loss = running_loss / total_batches
    epoch_accuracy = (running_accuracy / total_batches) * 100
    print(f"\nEpoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%")

    # Guardar el modelo si la pérdida ha disminuido
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), model_path)
        print(f"Modelo guardado en la época {epoch+1} con pérdida de {epoch_loss:.4f}.")

print("Entrenamiento finalizado.")


KeyboardInterrupt: 

In [ ]:
from torch.utils.data import DataLoader

# Path del dataset de Test
test_image_dir = '/content/drive/MyDrive/TestOjos'

# Las imagenes se transforman segun la configuracion del modelo
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Cargar el dataset
test_dataset = datasets.ImageFolder(root=test_image_dir, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Poner el modelo en modo de evaluación
model.eval()
model.to(device)

# Función para realizar las predicciones
def predict_with_outputs_and_names(model, dataloader, dataset):
    predictions = []
    labels = []
    outputs_list = []  # Para almacenar las salidas crudas (logits)
    image_names = []   # Para almacenar los nombres de las imágenes

    with torch.no_grad():
        for i, (images, label) in enumerate(dataloader):
            images = images.to(device)
            outputs = model(images)
            outputs_list.extend(outputs.cpu().numpy())  # Guardar las salidas
            _, preds = torch.max(outputs, 1)  # Obtener las clases predichas
            predictions.extend(preds.cpu().numpy())
            labels.extend(label.numpy())

            # Obtener los nombres de las imágenes desde el dataset
            start_idx = i * dataloader.batch_size
            end_idx = start_idx + len(images)
            image_names.extend([dataset.imgs[idx][0] for idx in range(start_idx, end_idx)])

    return predictions, labels, outputs_list, image_names

# Realizar predicciones en el dataset de test y obtener las salidas
predictions, true_labels, outputs_list, image_names = predict_with_outputs_and_names(model, test_dataloader, test_dataset)

# Mostrar las salidas
for idx in range(len(image_names)):
    print(f"  Imagen: {image_names[idx]}")
    print(f"  Salida del modelo: {outputs_list[idx]}")
    print(f"  Predicción: {predictions[idx]} - Etiqueta Verdadera: {true_labels[idx]}")
    print("---------------------------------------------------------------------------")


  Imagen: /content/drive/MyDrive/TestOjos/Mild/11505_left.jpeg
  Salida del modelo: [1.8430984  1.02328    1.5348706  0.2960907  0.36236632]
  Predicción: 0 - Etiqueta Verdadera: 0
---------------------------------------------------------------------------
  Imagen: /content/drive/MyDrive/TestOjos/Mild/11505_right.jpeg
  Salida del modelo: [1.6659739  0.98677135 1.557403   0.1618863  0.34237123]
  Predicción: 0 - Etiqueta Verdadera: 0
---------------------------------------------------------------------------
  Imagen: /content/drive/MyDrive/TestOjos/Mild/11959_left.jpeg
  Salida del modelo: [1.6561369  0.79577696 1.7407235  0.36317644 0.50568485]
  Predicción: 2 - Etiqueta Verdadera: 0
---------------------------------------------------------------------------
  Imagen: /content/drive/MyDrive/TestOjos/Mild/11959_right.jpeg
  Salida del modelo: [1.508803   0.8847409  1.630891   0.13751367 0.6142112 ]
  Predicción: 2 - Etiqueta Verdadera: 0
----------------------------------------------

In [ ]:
from sklearn.metrics import accuracy_score
from collections import defaultdict
import numpy as np

# Función para calcular la precisión por clase
def accuracy_per_class(predictions, true_labels, num_classes):
    correct_counts = defaultdict(int)  # Correct predictions per class
    total_counts = defaultdict(int)    # Total predictions per class

    for pred, true in zip(predictions, true_labels):
        total_counts[true] += 1
        if pred == true:
            correct_counts[true] += 1

    # Calcular porcentaje de precisión por clase
    accuracy_per_class = {}
    for cls in range(NUM_CLASES):
        if total_counts[cls] > 0:
            accuracy_per_class[cls] = (correct_counts[cls] / total_counts[cls]) * 100
        else:
            accuracy_per_class[cls] = 0.0  # Si no hay ejemplos de la clase, precisión es 0

    return accuracy_per_class

# Obtener el porcentaje de aciertos por clase
accuracy_per_class_results = accuracy_per_class(predictions, true_labels, NUM_CLASES)

# Mostrar resultados de precisión por clase
for cls, acc in accuracy_per_class_results.items():
    print(f"Clase {cls}: {acc:.2f}% de aciertos")


Clase 0: 90.91% de aciertos
Clase 1: 0.00% de aciertos
Clase 2: 5.26% de aciertos
Clase 3: 0.00% de aciertos
Clase 4: 0.00% de aciertos
